# SQL I

Introducing SQL and databases.

## Starting Up SQL

Before we look at SQL syntax in detail, let's first get ourselves set up to run SQL queries in Jupyter.

#### Approach #1: SQL Magic

**1. Load the `sql` Module.**

Load `%%sql` cell magic.

In [1]:
%load_ext sql

**2. Connect to a database.**  

Here, we connect to the SQLite database `basic_examples.db`.

In [2]:
%%sql
sqlite:////content/basic_examples.db

<br/>

**3. Run a simple SQL query.**

Note the `%%sql` lets Jupyter parse the rest of the lines as a SQL command.

In [3]:
%%sql
SELECT * FROM Dragon;

 * sqlite:////content/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


**Simple query, this time on two different lines.**

In [4]:
%%sql
SELECT *
FROM Dragon;

 * sqlite:////content/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


#### Approach #2: `pd.read_sql`

It turns out that `pandas` has a special-purpose function to parse SQL queries. We can pass in a SQL query as a string to return a `pandas` DataFrame. To achieve the same result as we did using cell magic above, we can do the following.

**1. Connect to a database**

In [11]:
import sqlalchemy
import pandas as pd

engine = sqlalchemy.create_engine('sqlite:////content/basic_examples.db')
connection = engine.connect()

**2. Run a simple SQL query**

In [12]:
query = """
SELECT *
FROM Dragon;
"""

pd.read_sql(query, engine)

,name,year,cute
0,hiccup,2010,10.0
1,drogon,2011,-100.0
2,dragon 2,2019,0.0
3,puff,2010,100.0
4,smaug,2011,NaN


## Tables and Schema

A **database** contains a collection of SQL **tables**. Let's connect to our "toy" database `basic_examples.db` and explore the tables it stores.

In [52]:
%%sql
sqlite:///content/basic_examples.db

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 146, in __init__
    self._dbapi_connection = engine.raw_connection()
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 3300, in raw_connection
    return self.pool.connect()
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 449, in connect
    return _ConnectionFairy._checkout(self)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 1263, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 712, in checkout
    rec = pool._do_get()
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/impl.py", line 179, in _do_get
    with util.safe_reraise():
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/util/langhelpers.py", line 146, in __exit__
    raise exc_value.with_traceback(exc_tb)
  Fi

In [14]:
%%sql
SELECT * FROM sqlite_master WHERE type='table'

 * sqlite:////content/basic_examples.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,7,"CREATE TABLE sqlite_sequence(name,seq)"
table,Dragon,Dragon,2,"CREATE TABLE Dragon ( name TEXT PRIMARY KEY, year INTEGER CHECK (year >= 2000), cute INTEGER)"
table,Dish,Dish,4,"CREATE TABLE Dish ( name TEXT PRIMARY KEY, type TEXT, cost INTEGER CHECK (cost >= 0))"
table,Scene,Scene,6,"CREATE TABLE Scene ( id INTEGER PRIMARY KEY AUTOINCREMENT, biome TEXT NOT NULL, city TEXT NOT NULL, visitors INTEGER CHECK (visitors >= 0), created_at DATETIME DEFAULT (DATETIME('now')))"


## Basic Queries

Every SQL query *must* contain a `SELECT` and `FROM` clause.

* `SELECT`: specify the column(s) to return in the output
* `FROM`: specify the database table from which to extract data

**Question:** Select all columns from the **Dragon** table.

In [15]:
%%sql
SELECT * FROM Dragon;

 * sqlite:////content/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


**Question:** Select columns **cute** and **year** from the **Dragon** table.

In [22]:
%%sql
SELECT cute,year FROM Dragon;

 * sqlite:////content/basic_examples.db
Done.


cute,year
10,2010
-100,2011
0,2019
100,2010
None,2011


**Aliasing** with `AS`

**Question:** Repeat the last exercise with aliasing.

In [23]:
%%sql
SELECT cute as c,year as y FROM Dragon;

 * sqlite:////content/basic_examples.db
Done.


c,y
10,2010
-100,2011
0,2019
100,2010
None,2011


**Uniqueness** with `DISTINCT`

**Question:** Select all the unique years in the **Dragon** table.

In [26]:
%%sql
SELECT DISTINCT year FROM Dragon;

 * sqlite:////content/basic_examples.db
Done.


year
2010
2011
2019


**Filtering** with `WHERE`

**Question:** Select the **name** and **year** columns from the **Dragon** table such that the cute value is greater than 0.

In [25]:
%%sql
SELECT name,year FROM Dragon WHERE cute<0

 * sqlite:////content/basic_examples.db
Done.


name,year
drogon,2011


**Question:** Select the **name**, **cute** and **year** columns from the **Dragon** table such that the cute value is greater than 0 or the year is greater than 2013.

In [27]:
%%sql
Select name, cute, year from Dragon where cute>0 or year>2013

 * sqlite:////content/basic_examples.db
Done.


name,cute,year
hiccup,10,2010
dragon 2,0,2019
puff,100,2010


**Question:** Select the **name** and **year** columns from the **Dragon** table such that the name is either 'puff' or 'hiccup'.

In [28]:
%%sql
Select name, year from Dragon where name =='puff' or name='hiccup'

 * sqlite:////content/basic_examples.db
Done.


name,year
hiccup,2010
puff,2010


**Question:** Get the name and cute value of all dragons whose cute value is not null.

In [30]:
%%sql
select name,cute from Dragon where cute is not null

 * sqlite:////content/basic_examples.db
Done.


name,cute
hiccup,10
drogon,-100
dragon 2,0
puff,100


**Ordering** data using `ORDER BY`

**Question:** Sort the **Dragon** table in descending order of cuteness.

In [32]:
%%sql
select * from Dragon order by cute desc

 * sqlite:////content/basic_examples.db
Done.


name,year,cute
puff,2010,100
hiccup,2010,10
dragon 2,2019,0
drogon,2011,-100
smaug,2011,None


**Restricting** output with `LIMIT` and `OFFSET`

**Question:** Query the first two rows of the **Dragon** table.

In [35]:
%%sql
select * from Dragon limit 2

 * sqlite:////content/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100


**Question:** Query the two rows after the first row of the **Dragon** table.

In [36]:
%%sql
select * from Dragon limit 2 offset 1

 * sqlite:////content/basic_examples.db
Done.


name,year,cute
drogon,2011,-100
dragon 2,2019,0


## Grouping Data with `GROUP BY`

**Question:** Get all rows and columns of the **Dish** table.

In [37]:
%%sql
select * from  Dish

 * sqlite:////content/basic_examples.db
Done.


name,type,cost
ravioli,entree,10
ramen,entree,13
taco,entree,7
edamame,appetizer,4
fries,appetizer,4
potsticker,appetizer,4
ice cream,dessert,5


A small note: the fact that `type` is highlighted in green below is a consequence of Jupyter assuming that we are writing Python code (where `type` is a built-in keyword). `type` does *not* have a special meaning in SQL, so the color below does not indicate any special functionality. When we run the cell, Jupyter realizes it should recognize the code as SQL.

**Question:** Select the **type** column of the **Dish** table.

In [39]:
%%sql
select type from Dish

 * sqlite:////content/basic_examples.db
Done.


type
entree
entree
entree
appetizer
appetizer
appetizer
dessert


**Question:** Get all the dish types using GROUP BY.

In [41]:
%%sql
select *  from Dish group by type

 * sqlite:////content/basic_examples.db
Done.


name,type,cost
edamame,appetizer,4
ice cream,dessert,5
ravioli,entree,10


**Question:** Query the total cost of each type of dish.

In [42]:
%%sql
select * , sum(cost) from Dish group by type

 * sqlite:////content/basic_examples.db
Done.


name,type,cost,sum(cost)
edamame,appetizer,4,12
ice cream,dessert,5,5
ravioli,entree,10,30


**Question:** Query the total cost, the minimum cost and the name of the most expensive dish of each type.

In [46]:
%%sql
select * , sum(cost) as total_cost,min(cost) as min_cost,max(cost) as most_expensive from Dish group by type order by most_expensive desc

 * sqlite:////content/basic_examples.db
Done.


name,type,cost,total_cost,min_cost,most_expensive
ramen,entree,13,30,7,13
ice cream,dessert,5,5,5,5
edamame,appetizer,4,12,4,4


**Question:** Count the number of rows in each year in the **Dragon** table.

In [47]:
%%sql
select * , count(year) from Dragon group by year

 * sqlite:////content/basic_examples.db
Done.


name,year,cute,count(year)
hiccup,2010,10,2
drogon,2011,-100,2
dragon 2,2019,0,1


**Question:** Count the number of rows - including the rows with NULLs - in each year in the **Dragon** table.

In [49]:
%%sql
select * , count(year) from Dragon group by year


 * sqlite:////content/basic_examples.db
Done.


name,year,cute,count(year)
hiccup,2010,10,2
drogon,2011,-100,2
dragon 2,2019,0,1
